In [1]:
import os, sys
sys.path.append(os.path.abspath(".."))

import requests
import scrapy
import time
import pandas as pd
from pprint import pprint
import re
import json
import datetime

from scrapers import staff

In [2]:
data = staff.main()

Thu Sep 23 19:13:09 2021
Starting crawling 'https://staff.am'.
1474 jobs detected.
Starting crawling job postings..


URL https://staff.am/en/onlayn-xagavar-29: 100%|██████████| 1474/1474 [50:27<00:00,  2.05s/it]


1474/1474 jobs scraped.
Saving files.
Company_Title: 1474
Company_URL: 1474
URL: 1474
Total_views: 1474
Followers: 1474
Active_Jobs: 1474
Jobs_History: 1474
Job_Views: 1474
Job_Title: 1474
Application_Deadline: 1474
Other: 1474
Industry: 1474
Employment_term: 1474
Job_Category: 1474
Job_type: 1474
Job_Location: 1474
Text: 1474
Salary: 1474
Job_description: 1474
Job_responsibilities: 1474
Required_qualifications: 1474
Required_candidate_level: 1474
Additional_information: 1474
Professional_skills: 1474
Soft_skills: 1474
Detected 15 new companies.
Starting crawling companies.


100%|██████████| 15/15 [00:12<00:00,  1.18it/s]

Extracted 15/15
Thu Sep 23 20:04:19 2021
Ended crawling 'https://staff.am'.


In [ ]:
len(data.keys())

In [ ]:
all_default_keys = (
        "Company_Title", "Company_URL", "URL", "Total_views", "Followers",
        "Active_Jobs", "Jobs_History", "Job_Views", "Job_Title",
        "Application_Deadline", "Industry", "Employment_term",
        "Job_Category", "Job_type", "Job_Location",
        "Job_description", "Job_responsibilities",
        "Required_qualifications", "Required_candidate_level",
        "Salary", "Additional_information",
        "Professional_skills", "Soft_skills"
    )

In [ ]:
new_keys = [k for k in data.keys() if k not in all_default_keys]

In [ ]:
len(new_keys)

In [ ]:
sorted([(k, len(data[k])) for k in new_keys], key=lambda x: x[1])[::-1]

In [ ]:
data["Additional_information"]

In [ ]:
rs = requests.get("https://staff.am/en/senior-uiux-designer-112")
response = scrapy.http.HtmlResponse(url=rs.url, body=rs.text, encoding="utf-8")

In [ ]:
staff.get_info_from_one_posting("https://staff.am/en/senior-uiux-designer-112")

In [ ]:
scrape = staff.crawl_all_postings(["https://staff.am/en/senior-engineer-ii-c", "https://staff.am/en/senior-uiux-designer-112"])

In [6]:
arr = [f"{i}th val" for i in range(100)]

In [7]:
from rich.progress import track
from rich import progress_bar

In [8]:
for i in track(arr):
    time.sleep(0.0001)

C:\Users\User\.virtualenvs\JobScraper-iUeExTuY\lib\site-packages\rich\live.py:227: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')
